In [ ]:
"""
Purpose: To decompose the multi-somas for splitting
using the new decomposition method



"""


In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

INFO - 2021-01-19 01:23:27,204 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-19 01:23:27,205 - settings - Setting database.user to celiib
INFO - 2021-01-19 01:23:27,206 - settings - Setting database.password to newceliipass
INFO - 2021-01-19 01:23:27,210 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-01-19 01:23:27,211 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-19 01:23:27,225 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-01-19 01:23:27,504 - settings - Setting enable_python_native_blobs to True


# Debugging the seg fault

In [2]:
#so that it will have the adapter defined
from datajoint_utils import *

In [3]:
test_mode = False

In [4]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-19 01:23:30,883 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-19 01:23:30,884 - settings - Setting database.user to celiib
INFO - 2021-01-19 01:23:30,886 - settings - Setting database.password to newceliipass
INFO - 2021-01-19 01:23:30,890 - settings - Setting enable_python_native_blobs to True


Sleeping 168 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-01-19 01:23:31,202 - settings - Setting enable_python_native_blobs to True


# Defining the Table

In [5]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [6]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 8546


In [10]:
decimation_version = 0
decimation_ratio = 0.25
# key_source = (minnie.Decimation().proj(decimation_version='version')  & 
#                   dict(decimation_version=decimation_version,decimation_ratio=decimation_ratio)  
#                   & minnie.MultiSomaProofread2() & (dj.U("segment_id") & (minnie.BaylorSegmentCentroid()).proj()))

key_source = ((minnie.Decimation).proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0") & minnie.MultiSomaProofread2())
key_source

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses)
864691134947393276,0,0.25
864691135012398582,0,0.25
864691135012471798,0,0.25
864691135012571126,0,0.25
864691135012905110,0,0.25
864691135065018692,0,0.25
864691135065024068,0,0.25
864691135065032772,0,0.25
864691135065033540,0,0.25
864691135081745143,0,0.25


In [ ]:
#schema.external['decomposition'].delete(delete_external_files=True)

In [12]:
import numpy as np
import time
decimation_version = 0
decimation_ratio = 0.25
process_version = 1

@schema
class Decomposition(dj.Computed):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    ver : decimal(6,2) #the version number of the materializaiton
    process_version : int unsigned #the version of the preprocessing pipeline run
    index : tinyint unsigned  #the index of the neuron object that resulted from that mesh (indexed starting at 0)
    ---
    multiplicity=null    : tinyint unsigned             # the number of somas found for this base segment
    decomposition: <decomposition>
    n_vertices           : int unsigned                 # number of vertices
    n_faces              : int unsigned                 # number of faces
    n_not_processed_soma_containing_meshes : int unsigned  #the number of meshes with somas that were not processed
    n_error_limbs: int #the number of limbs that are touching multiple somas or 1 soma in multiple places
    n_same_soma_multi_touching_limbs: int # number of limbs that touch the same soma multiple times
    n_multi_soma_touching_limbs: int # number of limbs that touch multiple somas
    n_somas: int #number of soma meshes detected
    n_limbs: int
    n_branches: int
    max_limb_n_branches=NULL:int
    
    skeletal_length=NULL: double
    max_limb_skeletal_length=NULL:double
    median_branch_length=NULL:double #gives information on average skeletal length to next branch point
    
    
    width_median=NULL: double #median width from mesh center without spines removed
    width_no_spine_median=NULL: double #median width from mesh center with spines removed
    width_90_perc=NULL: double # 90th percentile for width without spines removed
    width_no_spine_90_perc=NULL: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint

    spine_density=NULL: double # n_spines/ skeletal_length
    spines_per_branch=NULL: double
    
    skeletal_length_eligible=NULL: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches=NULL: int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible=NULL:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible=NULL:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume=NULL: double # the sum of all spine volume
    spine_volume_median=NULL: double # median of the spine volume for those spines with able to calculate volume
    spine_volume_density=NULL: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible=NULL: double #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible=NULL: double #total_spine_volume/n_spine_eligible_branches
    
    run_time=NULL : double                   # the amount of time to run (seconds)

    
    """


    
    key_source = ((minnie.Decimation).proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0") & minnie.MultiSomaProofread2())
                                                                  
    

    def make(self,key):
        """
        Pseudocode for process:

        1) Get the segment id from the key
        2) Get the decimated mesh
        3) Get the somas info
        4) Run the preprocessing
        5) Calculate all starter stats
        6) Save the file in a certain location
        7) Pass stats and file location to insert
        """
        whole_pass_time = time.time()
        #1) Get the segment id from the key
        segment_id = key["segment_id"]
        description = str(key['decimation_version']) + "_25"
        print(f"\n\n\n---- Working on Neuron {key['segment_id']} ----")
        global_start = time.time()
        
        #2) Get the decimated mesh
        current_neuron_mesh = du.fetch_segment_id_mesh(segment_id)
        

        #3) Get the somas info 
        somas = du.get_soma_mesh_list(segment_id) 
        soma_ver = du.get_soma_mesh_list_ver(segment_id)
                  
        
        print(f"somas = {somas}")
                  
        #3b) Get the glia and nuclei information 
        glia_faces,nuclei_faces = du.get_segment_glia_nuclei_faces(segment_id,return_empty_list=True)
        
                  
        #4) Run the preprocessing


        total_neuron_process_time = time.time()

        print(f"\n--- Beginning preprocessing of {segment_id}---")
        recovered_neuron = neuron.Neuron(
        mesh = current_neuron_mesh,
        somas = somas,
        segment_id=segment_id,
        description=description,
        suppress_preprocessing_print=False,
        suppress_output=False,
        calculate_spines=True,
        widths_to_calculate=["no_spine_median_mesh_center"],
        glia_faces=glia_faces,
        nuclei_faces = nuclei_faces,

                )

        print(f"\n\n\n---- Total preprocessing time = {time.time() - total_neuron_process_time}")


        #5) Don't have to do any of the processing anymore because will do in the neuron object
        stats_dict = recovered_neuron.neuron_stats()



        #6) Save the file in a certain location
        save_time = time.time()
        ret_file_path = recovered_neuron.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                          return_file_path=True,
                                         export_mesh=False,
                                         suppress_output=True)

        ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
        print(f"Save time = {time.time() - save_time}")

        #7) Pass stats and file location to insert
        new_key = dict(key,
                       ver = soma_ver,
                       process_version = process_version,
                       index = 0,
                       multiplicity=1,
                       decomposition=ret_file_path_str,
                       n_vertices=len(current_neuron_mesh.vertices),
                       n_faces=len(current_neuron_mesh.faces),
                       run_time=np.round(time.time() - whole_pass_time,4)
                      )
        new_key.update(stats_dict)

        self.insert1(new_key, allow_direct_insert=True, skip_duplicates=True)

        print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")
    

# Running the Populate

In [13]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition'")
(curr_table)#.delete()# & "status='error'")
#curr_table.delete()
#(curr_table & "error_message = 'ValueError: need at least one array to concatenate'").delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__decomposition,000f9dfbd04ba35b5b9547d6409fa6fa,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.128,at-node1,20,45341,2021-01-16 17:53:12
__decomposition,00150ca2e900357457919c067f8c9039,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.170,at-node43,19,43252,2021-01-15 22:05:27
__decomposition,004aa12704057ef24f96399aafe7de78,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.83,at-compute004,19,47761,2021-01-18 19:16:21
__decomposition,006d4a754852db78637a9730066c3558,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.157,at-node30,20,49732,2021-01-18 02:09:08
__decomposition,00ed7695cf1aee03ce686b97931ac3d8,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.81,at-compute003,19,51191,2021-01-18 19:07:29
__decomposition,01057efbeb13fdb2d180fb5009bd8513,reserved,=BLOB=,,=BLOB=,celiib@10.66.4.27,at-compute005,19,43068,2021-01-16 04:35:47
__decomposition,010b9ec46aebb5fbb96e72bcf98b3330,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.142,at-node15,19,51692,2021-01-18 18:10:09
__decomposition,0110368ebd0591c101fc86dbc80dcfc2,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.145,at-node18,19,48166,2021-01-17 12:06:25
__decomposition,013cf0866886985921c61b7ca4165297,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.136,at-node9,20,44208,2021-01-16 01:26:21
__decomposition,0143e4277fc86c812cf66933ff2b29cd,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.167,at-node40,23,48161,2021-01-17 04:38:09


In [ ]:
import time
import random
import compartment_utils as cu
cu = reload(cu)
import preprocessing_vp2 as pre
pre = reload(pre)
import skeleton_utils as sk
sk = reload(sk)
nru = reload(nru)

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    Decomposition.populate(reserve_jobs=True, suppress_errors=True)
else:
    Decomposition.populate(reserve_jobs=True, suppress_errors=False)
print('Populate Done')

print(f"Total time for DecompositionMultiSoma populate = {time.time() - start_time}")

INFO - 2021-01-19 01:25:57,680 - autopopulate - Found 97 keys to populate
INFO - 2021-01-19 01:25:57,724 - connection - Transaction started
INFO - 2021-01-19 01:25:57,726 - autopopulate - Populating: {'segment_id': 864691135771720779, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25')}


Populate Started



---- Working on Neuron 864691135771720779 ----
somas = [[<trimesh.Trimesh(vertices.shape=(22615, 3), faces.shape=(44846, 3))>, <trimesh.Trimesh(vertices.shape=(17412, 3), faces.shape=(33070, 3))>], array([923.0204, 923.0204]), array([0.736, 0.827])]

--- Beginning preprocessing of 864691135771720779---
--- 0) Having to preprocess the Neuron becuase no preprocessed data
Please wait this could take a while.....
Skipping the hole filling
use_meshafterparty = True
Using pre-computed somas: soma_mesh_list = [<trimesh.Trimesh(vertices.shape=(22615, 3), faces.shape=(44846, 3))>, <trimesh.Trimesh(vertices.shape=(17412, 3), faces.shape=(33070, 3))>]
Soma List = [<trimesh.Trimesh(vertices.shape=(22615, 3), faces.shape=(44846, 3))>, <trimesh.Trimesh(vertices.shape=(17412, 3), faces.shape=(33070, 3))>]
soma_mesh_list_centers = [array([ 738503.34762768,  514683.23853637, 1020524.32580146]), array([ 791692.112187  ,  430339.90751781, 1095853.07035378])]
Getting Glia and Nuclei Pi

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:08<00:00,  8.97s/it]

meshparty_segment_size = 100
Time for 1st pass MP skeletonization: 11.40138030052185
connecting at the root
branches_touching_root = [76]
length of Graph = 26599
Working on path [10710. 10723. 10730.]
path_degrees = [3, 2, 3]
Working on path [16827. 16869.]
path_degrees = [3, 3]
Working on path [23472. 23487. 23501. 23515. 23531. 23547. 23567.]
path_degrees = [3, 2, 2, 2, 2, 2, 3]
After combining close endpoints max(kept_branches_idx) = 76, len(kept_branches_idx) = 74
empty_indices % = 0.0
 conflict_indices % = 0.007152448067961421



AFTER face_lookup_resolved_test



Decomposing first pass: 29.429062128067017
Attempting to use MeshAfterParty Skeletonization and Mesh Correspondence
Another print
Found len(mesh_large_idx) MAP candidates: [22379, 5860, 12389, 2133, 5182, 4809, 16210, 5188, 2991, 13446, 5995, 1822, 4496, 22819, 2182, 10488, 526, 220, 21791, 2322, 5625, 7047, 111, 13919, 9011, 10418, 20705, 526, 27106, 12242, 26218, 11127, 20770, 7513]
mesh_large_connectivity: 1.985288143157959
Finding MAP candidates connected components: 0.0003764629364013672
len(filtered_pieces) = 4
skeleton_connectivity_MP : 0.6386687755584717
Grouping MP Sublimbs by Graph: 0.13337326049804688
Divinding into MP and MAP pieces: 1.1920928955078125e-06
--- Working on MAP piece 0---
MAP Filtering Soma Pieces: 0.019891023635864258
inside skeletonize_connected_branch and use_surface_after_CGAL=False, surface_reconstruction_size=1000
     Starting Screened Poisson
xvfb-run -n 8546 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Auto_Proofreading/Preprocessing_Pip